In [ ]:
import geemap
import ee

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="ee-zhanchaoyang")

In [ ]:
m = geemap.Map(center=(40, -100), zoom=4)
m

In [ ]:
geometry = ee.Geometry.Polygon(
    [[74.322, 14.981], [74.322, 14.765], [74.648, 14.765], [74.648, 14.980]]
)

In [ ]:
m.add_basemap("SATELLITE")

In [ ]:
m.addLayer(geometry, {}, "geometry")
m.centerObject(geometry)

In [ ]:
startDate = ee.Date.fromYMD(2022, 1, 1)

In [ ]:
endDate = ee.Date.fromYMD(2023, 1, 1)

In [ ]:
embeddings = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

In [ ]:
embeddingsFiltered = embeddings.filter(ee.Filter.date(startDate, endDate)).filter(
    ee.Filter.bounds(geometry)
)

## Visulization

In [ ]:
embeddings_vis = embeddingsFiltered.median().clip(geometry)

In [ ]:
vis_params = {
    "bands": ["A00", "A01", "A02"],
    "min": -0.5,
    "max": 0.5,
}

In [ ]:
m.addLayer(embeddings_vis, vis_params, "embeddings")
m

## Tasks continue

In [ ]:
# check the projections
embeddingsProjection = ee.Image(embeddingsFiltered.first()).select(0).projection()

In [ ]:
embeddingsProjection

In [ ]:
embeddingsImage = embeddingsFiltered.mosaic().setDefaultProjection(embeddingsProjection)

In [ ]:
gedi = ee.ImageCollection("LARSE/GEDI/GEDI04_A_002_MONTHLY")

In [ ]:
def qualityMask(image):
    mask = image.updateMask(image.select("l4_quality_flag").eq(1)).updateMask(
        image.select("degrade_flag").eq(0)
    )
    return mask

In [ ]:
def errorMask(image):
    relative_se = image.select("agbd_se").divide(image.select("agbd"))
    error_mask = image.updateMask(relative_se.lte(0.5))
    return error_mask

In [ ]:
def SlopeMask(image):
    glo30 = ee.ImageCollection("COPERNICUS/DEM/GLO30")
    glo30Filtered = glo30.filter(ee.Filter.bounds(geometry)).select("DEM")
    demProj = glo30Filtered.first().select(0).projection()
    elevation = glo30Filtered.mosaic().rename("dem").setDefaultProjection(demProj)
    slope = ee.Terrain.slope(elevation)
    slope_mask = image.updateMask(slope.lte(30))
    return slope_mask

In [ ]:
gediFiltered = gedi.filter(ee.Filter.date(startDate, endDate)).filter(
    ee.Filter.bounds(geometry)
)

In [ ]:
gediProjection = ee.Image(gediFiltered.first()).select("agbd").projection()

In [ ]:
gedi_processed = gediFiltered.map(qualityMask).map(errorMask).map(SlopeMask)

In [ ]:
gediMosaic = gedi_processed.mosaic().select("agbd").setDefaultProjection(gediProjection)

In [ ]:
gediVis = {
    min: 0,
    max: 200,
    "palette": ["#edf8fb", "#b2e2e2", "#66c2a4", "#2ca25f", "#006d2c"],
    "bands": ["agbd"],
}

In [ ]:
m.addLayer(gedi_processed, gediVis, "gedi")
m

In [ ]:
gridScale = 100

In [ ]:
gridProjection = ee.Projection("EPSG:3857").atScale(gridScale)

In [ ]:
stacked = embeddingsImage.addBands(gediMosaic)

In [ ]:
stacked = stacked.resample("bilinear")

In [ ]:
stackedResampled = stacked.reduceResolution(
    reducer=ee.Reducer.mean(), maxPixels=1024
).reproject(crs=gridProjection)

In [ ]:
stackedResampled = stackedResampled.updateMask(stackedResampled.mask().gt(0))

In [ ]:
export_image = stackedResampled.clip(geometry)

In [ ]:
geemap.ee_export_image_to_drive(
    export_image,
    description="GEDI_Mosaic_Export",
    folder="EarthEngine",  # Change to your Google Drive folder
    fileNamePrefix="gedi_mosaic",  # No ".tif" extension
    scale=gridScale,
    region=geometry,
    maxPixels=1e10,
)

In [ ]:
predictors = embeddingsImage.bandNames()

In [ ]:
predicted = gediMosaic.bandNames().get(0)

In [ ]:
print("predictors", predictors.getInfo())

In [ ]:
print("predicted", predicted.getInfo())

In [ ]:
predictorImage = stackedResampled.select(predictors)

In [ ]:
predictedImage = stackedResampled.select([predicted])

In [ ]:
classMask = predictedImage.mask().toInt().rename("class")

In [ ]:
numSamples = 1000

In [ ]:
training = stackedResampled.addBands(classMask).stratifiedSample(
    numPoints=numSamples,
    classBand="class",
    region=geometry,
    scale=gridScale,
    classValues=[0, 1],
    classPoints=[0, numSamples],
    dropNulls=True,
    tileScale=16,
)

In [ ]:
model = (
    ee.Classifier.smileRandomForest(50)
    .setOutputMode("REGRESSION")
    .train(features=training, classProperty=predicted, inputProperties=predictors)
)

In [ ]:
predicted = training.classify(classifier=model, outputName="agbd_predicted")

In [ ]:
def calculatermse(inputs):
    observed = ee.Array(inputs.aggregate_array("agbd"))
    predicted = ee.Array(inputs.aggregate_array("agbd_predicted"))
    rmse = (
        observed.subtract(predicted)
        .pow(2)
        .reduce(ee.Reducer.mean(), [0])
        .sqrt()
        .get([0])
    )
    return rmse

In [ ]:
rmse = calculatermse(predicted)
print("RMSE", rmse.getInfo())

## Plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt